# Neural Network Regressor for Expected Goal (xG) Modeling

To further extend the modeling pipeline, we train a **Neural Network (NN) Regressor** on dataset DS4.  

Unlike tree-based models, Neural Networks can learn **complex non-linear interactions** between features through multiple hidden layers and activation functions. They are particularly effective when relationships among features are subtle or high-dimensional, although they require more careful **regularization** to avoid overfitting.  

The chosen NN architecture includes: 

- **Input layer** matching the number of features.  

- **Hidden layers** with ReLU activation to capture non-linear patterns.  

- **Dropout layers** for regularization and overfitting prevention.  

- **Output layer** with a single neuron to predict the continuous xG value in [0,1].  

The model is trained using the **Adam optimizer** and **Mean Squared Error (MSE)** loss, with **early stopping** applied to prevent overfitting.  

#### Imports and Global Settings

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split

import os
import random

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)

# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

# Output paths
OUTPUT_DIR = "../task1_xg/outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Model directory
MODEL_DIR = "../task1_xg/models"
os.makedirs(MODEL_DIR, exist_ok=True)

print("Setup complete. Ready to load data.")


Setup complete. Ready to load data.


#### Load Dataset DS4

In [ ]:
DATA_PATH = "../task1_xg/data/DS4.csv" 
ds4 = pd.read_csv(DATA_PATH)

# Drop ID columns since they are not needed for modeling
ds4 = ds4.drop(columns=['event_id', 'match_id', 'player_id'])

print(f"Dataset loaded: {ds4.shape[0]} rows, {ds4.shape[1]} columns")
ds4.head()

Dataset loaded: 86833 rows, 26 columns


,minute,second,period,shot_type,shot_technique,shot_body_part,play_pattern,under_pressure,shot_first_time,shot_one_on_one,target_xg,loc_x,loc_y,end_shot_x,end_shot_y,end_shot_z,end_shot_z_available,shot_from_set_piece,distance_to_goal,angle_to_goal,gender,role,num_players_between,closest_defender_dist,goalkeeper_positioning,free_proj_goal
0,0.0435,0.8136,2,3,1,3,2,False,True,False,0.0566,0.7774,0.4366,0.6561,0.4393,0.0000,False,False,0.2143,0.1293,1,3,0.3077,0.0104,0.0132,0.7132
1,0.0507,0.6780,2,3,1,1,1,True,True,False,0.1434,0.9347,0.4166,0.9645,0.4456,0.1364,True,False,0.0871,0.2250,1,0,0.0769,0.0079,0.0123,0.3670
2,0.0797,0.1356,2,3,1,1,2,False,True,False,0.0382,0.8416,0.6964,0.8766,0.5845,0.0000,False,False,0.2227,0.0865,1,0,0.1538,0.0329,0.0068,0.5547
3,0.0942,0.2712,2,3,1,0,2,False,False,False,0.0528,0.9269,0.5910,0.8897,0.5845,0.0000,False,False,0.0995,0.1940,1,0,0.0769,0.0071,0.0045,0.5316
4,0.1159,0.0000,2,3,1,1,1,True,False,False,0.0213,0.6534,0.5295,0.6523,0.5156,0.0000,False,False,0.3301,0.0872,1,3,0.2308,0.0208,0.0019,0.7198


####  Define features, target and train/test split

In [15]:
target_column = "target_xg"  # expected goals column
train_columns = [col for col in ds4.columns if col != target_column]

X = ds4[train_columns]
y = ds4[target_column]

# Sanity check on target
print("\nTarget (xG) stats:")
print(y.describe())
print(f"Range: {y.min():.4f} - {y.max():.4f}")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"\nTraining set: {X_train.shape[0]} rows")
print(f"Test set: {X_test.shape[0]} rows")


Target (xG) stats:
count   86,833.0000
mean         0.0973
std          0.1281
min          0.0002
25%          0.0275
50%          0.0540
75%          0.1060
max          0.9951
Name: target_xg, dtype: float64
Range: 0.0002 - 0.9951

Training set: 69466 rows
Test set: 17367 rows


#### Training the Neural Network

To extend the analysis to a neural approach, a **Multi-Layer Perceptron (MLP) Regressor** is trained on dataset DS4.  

A **RandomizedSearchCV with 5-fold cross-validation** is employed to efficiently identify the best hyperparameters, optimizing for **Root Mean Squared Error (RMSE)** as the main evaluation metric.  

Unlike GridSearchCV, RandomizedSearchCV samples only a fixed number of configurations from the defined search space. This approach makes the process faster and computationally lighter, while still exploring a diverse range of parameter combinations.  
It therefore represents a **trade-off between exploration and efficiency**: the absolute best configuration may not be guaranteed, but the method provides a very good solution with far fewer evaluations, saving substantial time and resources.  

The hyperparameters explored include:  

- `hidden_layer_sizes`: architecture of the network, defining the number of neurons per hidden layer.  

- `activation`: non-linear transformation applied at each neuron (`relu` or `tanh`).  

- `alpha`: L2 regularization parameter, preventing overfitting by penalizing large weights.  

- `learning_rate_init`: initial learning rate controlling the step size of weight updates.  

- `solver`: optimization algorithm used for training (`adam` or `lbfgs`).  

At the end of the search, the best model (`best_mlp`) is automatically retrained on the full training set with the selected hyperparameters, and is then used for prediction and evaluation on both the training and test sets.

In [16]:
from sklearn.model_selection import RandomizedSearchCV

# Base model
mlp = MLPRegressor(
    max_iter=1000,                 # Maximum number of iterations
    early_stopping=True,           # Stop training when validation score is not improving
    random_state=RANDOM_STATE      # Reproducibility
)

# Parameter grid
param_dist = {
    "hidden_layer_sizes": [(64,), (128,), (64, 32), (128, 64)],
    "activation": ["relu", "tanh"],
    "alpha": [1e-5, 1e-4, 1e-3],          # L2 regularization
    "learning_rate_init": [0.001, 0.01],  # initial learning rate
    "solver": ["adam"]           # optimizers
}

# RandomizedSearchCV
rand_search = RandomizedSearchCV(
    estimator=mlp,                          # MLP model
    param_distributions=param_dist,         # parameter distributions for hyperparameter tuning
    n_iter=100,                             # number of random combinations to try
    scoring="neg_root_mean_squared_error",  # RMSE as main metric (neg because of sklearn convention)
    cv=5,                                   # cross-validation splitting strategy
    random_state=RANDOM_STATE,              # reproducibility
    n_jobs=-1,                              # parallelization
    verbose=2                               # verbosity level, defines the amount of information displayed during training
)

# Fit
rand_search.fit(X_train, y_train)

# Best model
best_mlp = rand_search.best_estimator_
print("\nBest parameters found:", rand_search.best_params_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.01, solver=adam; total time=  22.6s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.01, solver=adam; total time=  24.4s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.001, solver=adam; total time=  36.5s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.01, solver=adam; total time=  37.3s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.01, solver=adam; total time=  16.6s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.001, solver=adam; total time=  45.9s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.01, solver=adam; total time=  23.3s
[CV] END activation=relu, alpha=1e-05, hidden_layer_sizes=(64,), learning_rate_init=0.001, solver=adam; total time=  59.6s
[CV] END activation=r

In [17]:
def evaluate_predictions(y_true, y_pred, model_name="Model"):
    """
    Evaluate regression performance using a set of metrics
    suitable for probabilistic xG estimation (continuous target in [0, 1]).

    Parameters
    ----------
    y_true : array-like
        Ground truth (true xG values).
    y_pred : array-like
        Predicted xG values.
    model_name : str
        Label to display in output.

    Returns
    -------
    metrics : dict
        Dictionary of metric values.
    """

    metrics = {
        # Root Mean Squared Error (RMSE):
        # sqrt( (1/n) * Σ (y_i - ŷ_i)^2 )
        # Penalizes large errors more strongly; same units as target.
        "RMSE": root_mean_squared_error(y_true, y_pred),

        # Mean Absolute Error (MAE):
        # (1/n) * Σ |y_i - ŷ_i|
        # Robust to outliers; easy to interpret as "average absolute error".
        "MAE": mean_absolute_error(y_true, y_pred),

        # Coefficient of Determination (R²):
        # 1 - (Σ (y_i - ŷ_i)^2) / (Σ (y_i - ȳ)^2)
        # Measures proportion of variance explained by the model.
        "R2": r2_score(y_true, y_pred),

        # Pearson Correlation Coefficient:
        # cov(y, ŷ) / (σ_y * σ_ŷ)
        # Measures strength of linear relationship between y_true and y_pred.
        "Pearson": pearsonr(y_true, y_pred)[0],

        # Spearman Rank Correlation:
        # Pearson correlation between rank(y) and rank(ŷ)
        # Captures monotonic (not necessarily linear) relationships.
        "Spearman": spearmanr(y_true, y_pred)[0]
    }

    # Print nicely formatted results
    print(f"\n{model_name} performance:")
    for k, v in metrics.items():
        print(f"{k:>12}: {v:.4f}")

    return metrics


In [ ]:
# Predict on train/test
y_train_pred = best_mlp.predict(X_train)
y_test_pred  = best_mlp.predict(X_test)

# Evaluate with the same function used across models
train_metrics = evaluate_predictions(y_train, y_train_pred, "Neural Network (train)")
test_metrics  = evaluate_predictions(y_test,  y_test_pred,  "Neural Network (test)")



Neural Network (train) performance:
        RMSE: 0.0368
         MAE: 0.0199
          R2: 0.9165
     Pearson: 0.9574
    Spearman: 0.9566

Neural Network (test) performance:
        RMSE: 0.0427
         MAE: 0.0217
          R2: 0.8936
     Pearson: 0.9456
    Spearman: 0.9554


#### Features Importance

Unlike Linear Regression, Random Forest, or XGBoost, a **Neural Network does not provide a straightforward feature importance measure**. The weights of the network cannot be directly interpreted as feature contributions, since they are distributed across multiple layers and transformed by non-linear activation functions.  

For consistency and interpretability, **feature importance will only be reported for models that natively support it** (Linear Regression, Random Forest, XGBoost). 

#### Save results and trained model

In [19]:
import joblib
import pandas as pd

# Save metrics
results_df = pd.DataFrame(
    [train_metrics, test_metrics],
    index=["Neural Network (train)", "Neural Network (test)"]
)
results_df.to_csv(f"{OUTPUT_DIR}/metrics_nn.csv", index=True)

# Save model
model_path = f"{MODEL_DIR}/model_nn.pkl"
joblib.dump(best_mlp, model_path)

print(f"Metrics saved to {OUTPUT_DIR}/metrics_nn.csv")
print(f"Model saved to {model_path}")


Metrics saved to ../task1_xg/outputs/metrics_nn.csv
Model saved to ../task1_xg/models/model_nn.pkl


## Conclusion

The **Neural Network (MLP Regressor)** achieved solid performance on both the training and test sets.  
On the **training set**, the model reached an **RMSE of 0.0368** and an **R² of 0.9165**, showing that it was able to capture most of the variance in the data.  
On the **test set**, the performance slightly decreased to an **RMSE of 0.0427** and an **R² of 0.8936**, indicating a little performance drop but still strong generalization capabilities.  

Correlation metrics further confirm the model’s robustness: both **Pearson (0.9456)** and **Spearman (0.9554)** remain high on the test set, suggesting that the Neural Network preserves the correct ranking of predictions and maintains consistency with the true target distribution.  

Overall, the Neural Network performs **competitively compared to ensemble methods (Random Forest, XGBoost)**, striking a good balance between training accuracy and test generalization, with only a modest degree of overfitting.
